### Plot derived variables against original data 
To run this Notebook:
- Install with instructions at https://bitbucket.org/mbari/auv-python
- `cd auv-python`
- `poetry install && poetry shell`
- `cd notebooks && jupyter notebook 2.0-mpm-quick-look-derived.ipynb`

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import pylab as plt
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f != '.DS_Store'])
display(auv_picker)

In [ ]:
# Pick auv mission on local system
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir))
                                                   if f != '.DS_Store'])
display(mission_picker)

In [ ]:
mission = mission_picker.value

# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
ncs = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        ncs[nc_file] = xr.open_dataset(os.path.join(netcdfs_dir, nc_file))
    except FileNotFoundError:
        pass
    
cal_align = f"{auv_name}_{mission}.nc"
ncs[cal_align] = xr.open_dataset(os.path.join(netcdfs_dir, cal_align))
display(ncs[cal_align])      # Uncomment to explore dataset

In [ ]:
%matplotlib notebook

plt.figure(figsize=(9,2))
plt.plot(ncs[cal_align]['depth_time'],
         ncs[cal_align]['depth_filtdepth'], 'b-',
         ncs[cal_align]['navigation_time'],
         ncs[cal_align]['navigation_depth'], 'y:o',
         ncs['parosci.nc']['time'],
         ncs['parosci.nc']['depth'], '.')
plt.legend(('depth_filtdepth', 'navigation_depth', 'parosci_depth'))
title = "Comparing Depth Variables"
plt.title(title)
plt.grid()

In [ ]:
# Print the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)